In [3]:
#This script sets up the environment for mouse metabolic modeling and data analysis.
#Importing necessary libraries for metabolic modeling, data manipulation, and visualization.

import cobra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
import os
os.getcwd()

'c:\\Users\\mabes\\Desktop\\GEM_PROJECT\\notebook'

In [4]:
#Loading the mouse genome-scale metabolic model (GEM).
model = cobra.io.read_sbml_model("../model/Mouse-GEM.xml")

https://identifiers.org/taxonomy/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


## Rationale for model choice
Mouse-GEM was selected because Mus musculus is a well-established model organism for studying mammalian metabolism and disease. Mice share substantial genetic, physiological, and metabolic similarity with humans, making them highly relevant for translational metabolic research.

Using a genome-scale metabolic model of the mouse allows the investigation of complex metabolic networks in a biologically meaningful yet computationally tractable framework. Insights gained from this model can help inform the understanding of human metabolic disorders and systemic metabolic behaviour.

# Questions to answer
1. Describe the underlying topology of the chosen GEM
2. Describe metabolic flux simulations using common objectives 
functions such as biomass production.
3. Describe how different model input (media/nutrients) impact 
model predictions.
4. Assess gene essentiality and which genes are important for model 
optimization

In [14]:
#Investigating the model structure and contents.
#Show all model attributes
model 

Name,MouseGEM
Memory address,2a86f1e96a0
Number of metabolites,8454
Number of reactions,12987
Number of genes,2846
Number of groups,151
Objective expression,1.0*MAR00021 - 1.0*MAR00021_reverse_97974
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [15]:
#Checking details of an individual reaction
model.reactions[890]

Reaction identifier,MAR06791
Name,4-Hydroxyphenylacetaldehyde:NADP+ oxidoreductase
Memory address,0x2a828199490
Stoichiometry,MAM01002c + MAM02040c + MAM02554c --> MAM01003c + 2.0 MAM02039c + MAM02555c 4-hydroxyphenylacetaldehyde + H2O + NADP+ --> 4-hydroxyphenylacetate + 2.0 H+ + NADPH
GPR,Aldh1a3 or Aldh3a1 or Aldh3b1 or Aldh3b2 or Aldh3b3
Lower bound,0.0
Upper bound,1000.0


# RESULTS: 1. UNDERLYING TOPOLOGY:

The Mouse-GEM genome-scale metabolic model represents a highly detailed reconstruction of mouse metabolism. The model comprises 12,987 reactions, 8,454 metabolites, and 2,846 genes, reflecting extensive coverage of metabolic processes and gene–protein–reaction associations.

The metabolic network is distributed across multiple cellular compartments, including the cytosol, mitochondria, endoplasmic reticulum, Golgi apparatus, lysosome, peroxisome, nucleus, and extracellular space, capturing the compartmentalised nature of eukaryotic metabolism. This multi-compartment structure enables the representation of intracellular transport processes and organelle-specific metabolic functions.

Collectively, these structural characteristics confirm that Mouse-GEM constitutes a comprehensive genome-scale framework suitable for constraint-based metabolic simulations and downstream analyses. 

And example of it , is the reaction 890 as shown in figure (), whch shown detailed stoichiometry, cofactor usage, and gene–protein–reaction associations, including the representation of isoenzymes through logical GPR rules.

In [17]:
#check all reactions in the model
for rct in model.reactions:
    print(rct.id, rct.reaction)


MAR03905 MAM01796c + MAM02552c --> MAM01249c + MAM02039c + MAM02553c
MAR03907 MAM01796c + MAM02554c --> MAM01249c + MAM02039c + MAM02555c
MAR04097 MAM01252c + MAM01371c + MAM01597c --> MAM01261c + MAM01334c + MAM02759c
MAR04099 MAM01252m + MAM01371m + MAM01597m --> MAM01261m + MAM01334m + MAM02759m
MAR04108 MAM01257c + MAM01597c --> MAM01261c + MAM01334c + MAM02039c
MAR04133 MAM01252c + MAM01371c + MAM02039c --> MAM01257c + MAM02759c
MAR04281 MAM02039x + MAM02553x + MAM02819x <=> MAM02403x + MAM02552x
MAR04388 MAM02039c + MAM02553c + MAM02819c <=> MAM02403c + MAM02552c
MAR04283 MAM01249c + MAM02040c + MAM02554c --> MAM01252c + 2.0 MAM02039c + MAM02555c
MAR08357 MAM01249m + MAM02040m + MAM02552m --> MAM01252m + 2.0 MAM02039m + MAM02553m
MAR04379 MAM01371c + MAM01845c --> MAM01285c + MAM01841c + MAM02039c
MAR04301 MAM01845c + MAM03130c --> MAM01841c + MAM02039c + MAM03106c
MAR04355 MAM01690c + MAM01785c --> MAM02883c
MAR04358 MAM01285c + MAM02039c + MAM02696c --> MAM01371c + MAM02819c
MA

In [18]:
#Extracting te Stoichiometric matrix of the model.
S = cobra.util.array.create_stoichiometric_matrix(model)

In [19]:
S

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0., -1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], shape=(8454, 12987))

In [20]:
#To better visulize the stoichiometric matrix, we can convert it to a DataFrame with reaction and metabolite labels.
#Using pandas
S_df = pd.DataFrame(S, 
                    index=[met.id for met in model.metabolites], 
                    columns=[rxn.id for rxn in model.reactions])
print(S_df)

           MAR03905  MAR03907  MAR04097  MAR04099  MAR04108  MAR04133  \
MAM00001c       0.0       0.0       0.0       0.0       0.0       0.0   
MAM00001e       0.0       0.0       0.0       0.0       0.0       0.0   
MAM00002c       0.0       0.0       0.0       0.0       0.0       0.0   
MAM00002e       0.0       0.0       0.0       0.0       0.0       0.0   
MAM00003c       0.0       0.0       0.0       0.0       0.0       0.0   
...             ...       ...       ...       ...       ...       ...   
MAM15006x       0.0       0.0       0.0       0.0       0.0       0.0   
MAM15007r       0.0       0.0       0.0       0.0       0.0       0.0   
MAM01368r       0.0       0.0       0.0       0.0       0.0       0.0   
MAM01104x       0.0       0.0       0.0       0.0       0.0       0.0   
MAM10018x       0.0       0.0       0.0       0.0       0.0       0.0   

           MAR04281  MAR04388  MAR04283  MAR08357  ...  MAR15002  MAR15003  \
MAM00001c       0.0       0.0       0.0      

2. Describe metabolic flux simulations using common objectives  functions such as biomass production.

Focus: Analyze fluxes using common objective functions.

Key Points to Cover:
>Biomass production and growth rate.
>Other objective functions (e.g., ATP yield).
>Flux Balance Analysis (FBA) results.

In [ ]:
# 2. Running FBA to check model functionality
# 2.1 First, we are going to check the current objective function
print(model.objective)

Maximize
1.0*MAR00021 - 1.0*MAR00021_reverse_97974


In [8]:
r =model.reactions.get_by_id("MAR00021")
print(r.reaction)

MAM01307c + MAM01365c + MAM01369c + MAM01370c + MAM01450c + MAM01451r + MAM01589c + MAM01602c + MAM01628c + MAM01721n + MAM01722n + MAM01974c + MAM01975c + MAM01986c + MAM02125c + MAM02184c + MAM02360c + MAM02392c + MAM02426c + MAM02471c + MAM02724c + MAM02733c + MAM02750c + MAM02770c + MAM02847c + MAM02896c + MAM02908c + MAM02993c + MAM03089c + MAM03101c + MAM03135c + MAM03161c --> MAM03970c


Here , the objective funtion define what the model is trying to optimise: Mouse-GEM is configured to maximise biomas production, which is characterised by the consumption of multiple metabolic precursors to produce a pseudo-biomass metabolite ("MAM03970c"), and was therefore used as a proxy for cellular growth.

# For Apendix

In [9]:
# 2.2 Now, we can perform FBA to find the optimal flux distribution (maximum growth rate)
solution = model.optimize()
solution.objective_value

4.262827945443598

Since the objective function of the model is biomass production,The FBA solution indicates that, given the current constraints, the network can sustain a maximum biomass production rate of 4.26 unit of biomass flux per hour

In [10]:
#2.3 Countin of active reactions.
#This allows us to see how much of metabolites is actually used to support growth
active_reactions = solution.fluxes[solution.fluxes != 0] #selecting only reactions with non-zero flux
len(active_reactions)

1186

Although the Mouse-GEM model comprises 12,987 reactions, only a subset (1188) carried non-zero flux under biomass optimisation, which is roughly ~9% of the network, indicating that growth is supported by a defined active metabolic network

In [11]:
#2.4 Identifying the the highest flux reactions
top_reactions = active_reactions.sort_values(ascending=False).head(10)
print(top_reactions)

MAR11942    1000.0
MAR04363    1000.0
MAR11941    1000.0
MAR09360    1000.0
MAR04583    1000.0
MAR02351    1000.0
MAR11395    1000.0
MAR09437    1000.0
MAR04919    1000.0
MAR09072    1000.0
Name: fluxes, dtype: float64


In [12]:
for rxn_id, flux in top_reactions.items():
    rxn = model.reactions.get_by_id(rxn_id)
    print(rxn_id, rxn.name, rxn.reaction, flux)

MAR11942 Exchange of Xylu_D MAM01759e <=>  1000.0000000000002
MAR04363 2-phospho-D-glycerate hydro-lyase (phosphoenolpyruvate-forming) MAM00674c <=> MAM02040c + MAM02696c 1000.0000000000002
MAR11941 Assumed Passive Diffusion into Extracellular Space MAM01759c <=> MAM01759e 1000.0000000000001
MAR09360  MAM02167e <=>  1000.0000000000001
MAR04583 S-Adenosyl-L-methionine:guanidinoacetate N-methyltransferase MAM01619c + MAM02039c + MAM02871c <=> MAM02036c + MAM02877c 1000.0000000000001
MAR02351 Facilitated Diffusion MAM02751c --> MAM02751e 1000.0
MAR11395 Adenosine Transport Cnt3 MAM01280e + MAM02039e <=> MAM01280c + MAM02039c 1000.0
MAR09437  MAM03118e <=>  1000.0
MAR04919  MAM01596c <=> MAM01596e 1000.0
MAR09072  MAM02751e <=>  1000.0


>These reactions are unconstrained and are being pushed to their maximum allowed flux, not because biology requires it, but because nothing stops it.

>Several reactions reach their maximum allowed flux (1000), indicating that they are not limiting the optimal growth solution and are effectively unconstrained under the current model setup

>Several reactions reach their maximum allowed flux (1000), indicating that they are unconstrained and not growth-limiting under the current model setup. Many of these reactions correspond to exchange and transport processes, which are expected to hit their bounds when no environmental or capacity constraints are imposed.

>The reactions with flux = 1000 are not biologically dominant pathways, but unconstrained exchange, transport, or non-limiting reactions hitting their imposed upper bounds

>Even though the system is designed to optimise biomass production, these reactions still run at their maximum value because the biomass-related constraints do not affect them. So Biomass optimisation does not constrain all reactions, only those that matter for achieving biomass

In [14]:
#2.5, now we will identify reactions with lowest fluxes(negative)
#Are reactions operating in both directions under the optimisation?
bottom_reactions = active_reactions.sort_values().head(10)
print(bottom_reactions)


MAR11349   -1000.0
MAR04584   -1000.0
MAR09838   -1000.0
MAR09927   -1000.0
MAR03959   -1000.0
MAR10366   -1000.0
MAR04582   -1000.0
MAR09383   -1000.0
MAR04652   -1000.0
MAR09079   -1000.0
Name: fluxes, dtype: float64


>Several reactions carry large negative fluxes (−1000), indicating that they are reversible and unconstrained under the current model setup. These reactions are not coupled to biomass production and therefore reach their imposed lower bounds without affecting the optimisation objective

In [15]:
# 2.6 we are now asking , instead of growing as fast as possible, what 
#is the system tries to maximise other functions, such as ATP production?

# -----TO DO SO -----
#First, we have to find a ATP producing reaction or hydroyses it.
#Second, list atp-producing reactions
#Third, check ATP maintenance reaction
#Fourth, run FBA with ATP as objective
#Finally, Compare ATP-maximisation vs biomass-maximisation flux distributions
##NOTE: In Mouse-GEM, i must search by IDs, annotations, or stoichiometry — not names.

# ----------First------
for met in model.metabolites:
    if "atp" in met.id.lower() or "atp" in met.name.lower():
        print(met.id, met.name, met.compartment)



MAM01371c ATP c
MAM01371g ATP g
MAM01371l ATP l
MAM01371m ATP m
MAM01371n ATP n
MAM01371x ATP x
MAM01371r ATP r
MAM01371e ATP e
MAM01642c dATP c
MAM01642m dATP m
MAM01642n dATP n


>ATP is represented in multiple cellular compartments in the Mouse-GEM model. For the purpose of defining an ATP-related objective function, cytosolic ATP (MAM01371c) is the most relevant metabolite, as it represents the primary cellular energy currency.

In [16]:
#-----Second, list atp-producing reactions----
atp_rxns = []

for rxn in model.reactions:
    if model.metabolites.get_by_id("MAM01371c") in rxn.metabolites:
        atp_rxns.append(rxn)

len(atp_rxns)

882

>This value 882 tell us that , There are 882 reactions in the Mouse-GEM model that involve cytosolic ATP (MAM01371c), this include all reactions where ATP appear, regardless of whether ATP is consumed, produced, exchanged or transported.

But most of this reactions consume ATP, not produce it , and that is why we must filter it

In [17]:
#-------Third, identify ATP-producing reactions------
atp = model.metabolites.get_by_id("MAM01371c") #metabolite

atp_producing_rxns = []

for rxn in atp_rxns:
    coeff = rxn.metabolites.get(atp, 0)
    if coeff > 0:
        atp_producing_rxns.append(rxn)

len(atp_producing_rxns)


44

>Only 44 reactions in the entire Mouse-GEM network directly produce cytosolic ATP.

but we can not put an ATP producing reaction as an objective because If you set one ATP-producing reaction as the objective:

The solver will:

Push that one reaction to its bound

Ignore the rest of metabolism

You get:

Futile cycles

Unrealistic solutions

ATP maintenance avoids this by:

Aggregating ATP demand

Forcing balanced ATP production

In [70]:
#------fourth, Look for an ATP maintenance / demand reaction------
for rxn in model.reactions:
    if "atp" in rxn.id.lower() or "atp" in rxn.name.lower():
        print(rxn.id, rxn.name)


MAR04133 ATP:acetate adenylyltransferase
MAR04379 ATP:D-fructose-6-phosphate 1-phosphotransferase
MAR04358 ATP:pyruvate 2-O-phosphotransferase
MAR04368 ATP:3-phospho-D-glycerate 1-phosphotransferase
MAR04394 ATP:D-glucose 6-phosphotransferase
MAR04414 ATP:alpha-D-galactose 1-phosphotransferase
MAR08761 ATP:D-tagatose 6-phosphotransferase
MAR00454 ATP:D-glyceraldehyde 3-phosphotransferase
MAR04310 ATP:D-fructose 1-phosphotransferase
MAR04319 ATP:D-fructose 6-phosphotransferase
MAR04402 ATP:6-deoxy-L-galactose 1-phosphotransferase
MAR04490 ATP:D-mannose 6-phosphotransferase
MAR04595 ATP:D-xylulose 5-phosphotransferase
MAR08726 ATP:D-ribulose 5-phosphotransferase
MAR04459 ATP:propanoate adenylyltransferase
MAR04460 ATP:propanoate adenylyltransferase
MAR04052 ATP:D-ribose-5-phosphate diphosphotransferase
MAR04350 ATP:D-ribose 5-phosphotransferase
MAR04476 ATP:D-gluconate 6-phosphotransferase
MAR04567 ATP:Sedoheptulose 7-phosphate 1-phosphotransferase
MAR09799 ATP:sedoheptulose 7-phosphate


As Mouse-GEM does not include a predefined ATP maintenance reaction, as is a more complex organism, so ATP-based FBA requires explicitly adding an ATP demand reaction before setting it as the objective.

In [18]:
# Fifth, Add ATP maintenance (ATPM) to Mouse-GEM

#Gettin the requiered metabolites ("MAM01371c" is ATP in cytosol)
atp = model.metabolites.get_by_id("MAM01371c")

#getting other energy metabolites in the cytosol

adp = [m for m in model.metabolites if m.name == "ADP" and m.compartment == "c"][0]
pi  = [m for m in model.metabolites if m.name in ["Phosphate", "Pi"] and m.compartment == "c"][0]
h   = [m for m in model.metabolites if m.name == "H+" and m.compartment == "c"][0]

#Creating the ATPM reaction
from cobra import Reaction

ATPM = Reaction("ATPM")
ATPM.name = "ATP maintenance demand"
ATPM.lower_bound = 0
ATPM.upper_bound = 1000

#Defining the reaction stoichiometry
ATPM.add_metabolites({
    atp: -1.0,
    adp: 1.0,
    pi: 1.0,
    h: 1.0
})

#Adding the reaction to the model
model.add_reactions([ATPM])

#verifying the addition
model.reactions.get_by_any("ATPM")

[<Reaction ATPM at 0x1d88e188c50>]

The steps taken were not unnecessary; they were required to verify the model structure, understand ATP metabolism, and justify the use of ATPM as an objective.
as the model already had it but it was not clrear from before.

In [19]:
# ---Now, view the ATP maintenance requirement reaction--
model.reactions.get_by_id("ATPM")

Reaction identifier,ATPM
Name,ATP maintenance demand
Memory address,0x1d88e188c50
Stoichiometry,MAM01371c --> MAM01285c + MAM02039c + MAM02751c ATP --> ADP + H+ + Pi
GPR,
Lower bound,0
Upper bound,1000


The ATPM reaction represents ATP hydrolysis for cellular maintenance (ATP → ADP + Pi + H⁺) and serves as an energy demand sink that enables biologically meaningful ATP optimisation in FBA

In [20]:
#--- now we can set the objective to ATPM and re-run FBA---
model.objective = "ATPM"
print(model.objective)
model.reactions.get_by_id("ATPM").upper_bound = 1000.

Maximize
1.0*ATPM - 1.0*ATPM_reverse_5b752


In [21]:
#Now can run FBA again
model.optimize().objective_value

1000.0

>Under the current constraints, the model can hydrolyse ATP as fast as you allow it to (up to 1000), so ATP production is not limited by anything in the network.
>ATP production is completely unconstrained by the current environment and network limits.this is expected in a default, unconstrained GEM

In [98]:
#3. Describe how different model input (media/nutrients) impact 
#First ,we will check the current exchange reactions in the model

ex_rxns = model.exchanges
rxns = [x.id for x in ex_rxns]
eq = [x.reaction for x in ex_rxns]
lb = [x.bounds[0] for x in ex_rxns]
ub = [x.bounds[1] for x in ex_rxns]
out = pd.DataFrame(index = rxns,data = {'equation' : eq,
                                        'lb' : lb,
                                        'ub' : ub})
out


,equation,lb,ub
MAR07108,MAM01374e <=>,-1000.0,1000.0
MAR07110,MAM02556e <=>,-1000.0,1000.0
MAR07112,MAM01296e <=>,-1000.0,1000.0
MAR07114,MAM03044e <=>,-1000.0,1000.0
MAR07116,MAM01403e <=>,-1000.0,1000.0
...,...,...,...
MAR10088,MAM10031e <=>,-1000.0,1000.0
MAR10089,MAM10032e <=>,-1000.0,1000.0
MAR10090,MAM10033e <=>,-1000.0,1000.0
MAR10091,MAM10034e <=>,-1000.0,1000.0


In [103]:
#3.1 inspecting model inputs (exchange reactions = nutrients)
import pandas as pd

# Get all exchange reactions
ex_rxns = model.exchanges

rxns = [rxn.id for rxn in ex_rxns]
eqs  = [rxn.build_reaction_string(use_metabolite_names=True) for rxn in ex_rxns]
lb   = [rxn.lower_bound for rxn in ex_rxns]
ub   = [rxn.upper_bound for rxn in ex_rxns]

exchange_table = pd.DataFrame(
    index=rxns,
    data={
        "equation": eqs,
        "lower_bound": lb,
        "upper_bound": ub
    }
)

exchange_table.head()


,equation,lower_bound,upper_bound
MAR07108,benzo[a]pyrene <=>,-1000.0,1000.0
MAR07110,naphthalene <=>,-1000.0,1000.0
MAR07112,aflatoxin B1 <=>,-1000.0,1000.0
MAR07114,trichloroethene <=>,-1000.0,1000.0
MAR07116,bromobenzene <=>,-1000.0,1000.0


In [ ]:
#exchange reactions that allow uptake (negative lower bound)
exchange_table[exchange_table["lower_bound"] < 0]


,equation,lower_bound,upper_bound
MAR07108,benzo[a]pyrene <=>,-1000.0,1000.0
MAR07110,naphthalene <=>,-1000.0,1000.0
MAR07112,aflatoxin B1 <=>,-1000.0,1000.0
MAR07114,trichloroethene <=>,-1000.0,1000.0
MAR07116,bromobenzene <=>,-1000.0,1000.0
...,...,...,...
MAR10088,tauro-omega-muricholic acid <=>,-1000.0,1000.0
MAR10089,tauro-beta-muricholic acid <=>,-1000.0,1000.0
MAR10090,murideoxycholic acid <=>,-1000.0,1000.0
MAR10091,taurodehydrocholic acid <=>,-1000.0,1000.0


>This table defines the growth medium of the model. Reactions with a negative lower bound allow nutrient uptake.

In [105]:
#3.2 Default environment — growth and exchange fluxes
# Running FBA with default inputs
default_solution = model.optimize()
default_growth = default_solution.objective_value

print(f"Default biomass growth rate: {default_growth:.4f}")


Default biomass growth rate: 1000.0000


In [106]:
#Uptake reactions profile
df_default = default_solution.to_frame()
df_default = df_default.loc[[rxn.id for rxn in model.exchanges]]
df_default = df_default[df_default["fluxes"] != 0]

df_default


,fluxes,reduced_costs
MAR09816,0.000267,0.0
MAR09035,4.135245,0.0
MAR09039,-20.056534,0.0
MAR09042,-47.406354,0.0
MAR09044,20.056534,0.0
...,...,...
MAR13061,0.003824,0.0
MAR13063,0.003824,0.0
MAR13067,-2.187986,0.0
MAR10024,3.646643,0.0


Negative fluxes indicate uptake, positive fluxes indicate secretion.
This shows what the model consumes and produces under default conditions.

In [ ]:
#finding glucose exhance reaction
for rxn in model.exchanges:
    if "glucose" in rxn.name.lower() or "o2" in rxn.name.lower() or "oxygen" in rxn.name.lower():
        print(rxn.id, rxn.name, rxn.reaction)



MAR01947 Exchange of Alpha-D-Glucose 1-Phosphate MAM01967e <=> 


In [ ]:
#Finding oxygen reactions
for met in model.metabolites:
    if met.formula == "O2" or "oxygen" in met.name.lower():
        print(met.id, met.name, met.compartment)


MAM02630c O2 c
MAM02631c O2- c
MAM02630g O2 g
MAM02630l O2 l
MAM02630m O2 m
MAM02631m O2- m
MAM02630n O2 n
MAM02631n O2- n
MAM02630x O2 x
MAM02631x O2- x
MAM02630r O2 r
MAM02630e O2 e
MAM02631e O2- e
MAM02630i O2 i


In [ ]:
#finding oxygen exhance reaction
o2_e = model.metabolites.get_by_id("MAM02630e")

for rxn in model.exchanges:
    if o2_e in rxn.metabolites:
        print(rxn.id, rxn.name, rxn.reaction)


MAR09048  MAM02630e <=> 


In [27]:
model.objective = "MAR00021"  # Biomass reaction in Mouse-GEM
print(model.objective)

Maximize
1.0*MAR00021 - 1.0*MAR00021_reverse_97974


In [28]:
#Now that we have identified each exchange reaction for glucose and oxygen,
#we can modify their uptake rates to see how they impact growth.

#Helper function to set uptake rates
def set_medium(model, uptake_dict):
    # Close all uptake reactions
    for rxn in model.exchanges:
        rxn.lower_bound = 0
    
    # Open selected nutrients
    for rxn_id, lb in uptake_dict.items():
        model.reactions.get_by_id(rxn_id).lower_bound = lb


In [ ]:
#Minimal medium with only glucose and oxygen
#Using:
# --Glucose-1-phosphate: MAR01947
# -Oxygen: MAR09048

minimal_model = model.copy()

minimal_medium = {
    "MAR01947": -10,   # Alpha-D-glucose-1-phosphate
    "MAR09048": -20    # Oxygen (O2)
}

set_medium(minimal_model, minimal_medium)

sol_min = minimal_model.optimize()
print(f"Minimal medium growth rate: {sol_min.objective_value:.4f}")


Minimal medium growth rate: 0.0000


In [31]:
rich_model = model.copy()

# allow all default uptakes
for rxn in rich_model.exchanges:
    if rxn.lower_bound < 0:
        rxn.lower_bound = -1000

sol_rich = rich_model.optimize()
print(f"Rich medium growth rate: {sol_rich.objective_value:.4f}")


Rich medium growth rate: 4.2628


In [34]:
#Checking if glucose is necessary for growth in the model

# Copy the model to avoid modifying the original
no_glucose_model = model.copy()

# Block glucose uptake only
no_glucose_model.reactions.get_by_id("MAR01947").lower_bound = 0


solution_no_glucose = no_glucose_model.optimize()
print(f"Growth rate without glucose: {solution_no_glucose.objective_value:.4f}")


Growth rate without glucose: 4.2628


Glucose is not neccesary for biomass production in this model

In [36]:
# Reset objective function to biomass production

model.objective = "MAR00021"  # Biomass reaction in Mouse-GEM
print(model.objective)



Maximize
1.0*MAR00021 - 1.0*MAR00021_reverse_97974


This confirms that the model is optimising biomass production. All subsequent FBA simulations therefore predict cell growth rate under different environmental conditions.

In [ ]:
#3.1 Inspecting model inputs (exchange reactions)


ex_rxns = model.exchanges

exchange_table = pd.DataFrame(
    index=[rxn.id for rxn in ex_rxns],
    data={
        "equation": [rxn.build_reaction_string(use_metabolite_names=True) for rxn in ex_rxns],
        "lower_bound": [rxn.lower_bound for rxn in ex_rxns],
        "upper_bound": [rxn.upper_bound for rxn in ex_rxns]
    }
)

exchange_table.head()


,equation,lower_bound,upper_bound
MAR07108,benzo[a]pyrene <=>,-1000.0,1000.0
MAR07110,naphthalene <=>,-1000.0,1000.0
MAR07112,aflatoxin B1 <=>,-1000.0,1000.0
MAR07114,trichloroethene <=>,-1000.0,1000.0
MAR07116,bromobenzene <=>,-1000.0,1000.0


In [126]:
exchange_table[exchange_table["lower_bound"] < 0]


,equation,lower_bound,upper_bound
MAR07108,benzo[a]pyrene <=>,-1000.0,1000.0
MAR07110,naphthalene <=>,-1000.0,1000.0
MAR07112,aflatoxin B1 <=>,-1000.0,1000.0
MAR07114,trichloroethene <=>,-1000.0,1000.0
MAR07116,bromobenzene <=>,-1000.0,1000.0
...,...,...,...
MAR10088,tauro-omega-muricholic acid <=>,-1000.0,1000.0
MAR10089,tauro-beta-muricholic acid <=>,-1000.0,1000.0
MAR10090,murideoxycholic acid <=>,-1000.0,1000.0
MAR10091,taurodehydrocholic acid <=>,-1000.0,1000.0


Exchange reactions with a negative lower bound allow metabolite uptake. This table defines the baseline growth medium used for aerobic simulations and provides context for interpreting growth and flux predictions.

# Aerobic vs anaerobic conditions

In [16]:
#--------------3.0 --------------#
#3.0 Reset objective function
#revious analyses changed the objective (e.g. ATP maintenance)
# Reset objective to biomass
model.objective = "MAR00021"
print(model.objective)


Maximize
1.0*MAR00021 - 1.0*MAR00021_reverse_97974


In [ ]:
#3.1 Inspecting model inputs: exchange reactions
#Exchange reactions define the environmental inputs and outputs of the model (nutrients, gases, by-products).
import pandas as pd

ex_rxns = model.exchanges

exchange_table = pd.DataFrame(
    index=[rxn.id for rxn in ex_rxns],
    data={
        "equation": [rxn.build_reaction_string(use_metabolite_names=True) for rxn in ex_rxns],
        "lower_bound": [rxn.lower_bound for rxn in ex_rxns],
        "upper_bound": [rxn.upper_bound for rxn in ex_rxns]
    }
)

exchange_table.head()


,equation,lower_bound,upper_bound
MAR07108,benzo[a]pyrene <=>,-1000.0,1000.0
MAR07110,naphthalene <=>,-1000.0,1000.0
MAR07112,aflatoxin B1 <=>,-1000.0,1000.0
MAR07114,trichloroethene <=>,-1000.0,1000.0
MAR07116,bromobenzene <=>,-1000.0,1000.0


In [18]:
exchange_table[exchange_table["lower_bound"] < 0]


,equation,lower_bound,upper_bound
MAR07108,benzo[a]pyrene <=>,-1000.0,1000.0
MAR07110,naphthalene <=>,-1000.0,1000.0
MAR07112,aflatoxin B1 <=>,-1000.0,1000.0
MAR07114,trichloroethene <=>,-1000.0,1000.0
MAR07116,bromobenzene <=>,-1000.0,1000.0
...,...,...,...
MAR10088,tauro-omega-muricholic acid <=>,-1000.0,1000.0
MAR10089,tauro-beta-muricholic acid <=>,-1000.0,1000.0
MAR10090,murideoxycholic acid <=>,-1000.0,1000.0
MAR10091,taurodehydrocholic acid <=>,-1000.0,1000.0


The default Mouse-GEM medium is highly permissive (many nutrients can be taken up).

In [ ]:
#3.2 Identifying key gas exchange reactions (O₂ and CO₂)
#environmental inputs (especially oxygen) affect predictions.

#Oxygen exchange reaction
o2_e = model.metabolites.get_by_id("MAM02630e")

for rxn in model.exchanges:
    if o2_e in rxn.metabolites:
        print(rxn.id, rxn.reaction)


MAR09048 MAM02630e <=> 


In [20]:
#carbon dioxide exchange reaction
co2_e = model.metabolites.get_by_id("MAM01596e")

for rxn in model.exchanges:
    if co2_e in rxn.metabolites:
        print(rxn.id, rxn.reaction)


MAR09058 MAM01596e <=> 


In [21]:
#3.3 Aerobic condition (baseline)
#This represents the reference condition where oxygen uptake is allowed
aerobic_model = model.copy()

sol_aerobic = aerobic_model.optimize()
aerobic_growth = sol_aerobic.objective_value

print(f"Aerobic growth rate: {aerobic_growth:.4f}")


Aerobic growth rate: 4.2628


Under the aerobic baseline condition, the model predicts a biomass growth rate of 4.26, representing the maximum achievable growth given the default environmental constraints. No additional constraints were imposed on oxygen or carbon dioxide exchange, meaning the model was allowed to freely consume and secrete metabolites according to its predefined exchange bounds. This condition therefore reflects the default, highly permissive medium of Mouse-GEM rather than a strictly defined biological aerobic environment. The result serves as a reference point for comparison with constrained (anaerobic) simulations.

In [22]:
#checking oxygen usage
sol_aerobic.fluxes["MAR09048"]


np.float64(0.0)

Oxygen is not use on this model for the biomas production

In [23]:
#3.4 Anaerobic condition (oxygen uptake blocked)
#To test the impact of oxygen availability on model predictions.
anaerobic_model = model.copy()

# Block oxygen uptake
anaerobic_model.reactions.get_by_id("MAR09048").lower_bound = 0

sol_anaerobic = anaerobic_model.optimize()
anaerobic_growth = sol_anaerobic.objective_value

print(f"Anaerobic growth rate: {anaerobic_growth:.4f}")


Anaerobic growth rate: 4.2628


In [24]:
#3.5 Comparing growth predictions
print("Growth comparison:")
print(f" Aerobic growth:   {aerobic_growth:.4f}")
print(f" Anaerobic growth: {anaerobic_growth:.4f}")


Growth comparison:
 Aerobic growth:   4.2628
 Anaerobic growth: 4.2628


Blocking oxygen uptake did not alter the predicted biomass growth rate, which remained at 4.26 under anaerobic conditions. This indicates that oxygen is not required to achieve maximal biomass production in the current Mouse-GEM configuration. The result suggests that the optimal growth solution relies on oxygen-independent metabolic pathways. Consequently, oxygen availability is not a growth-limiting input under the default model constraints.

In [ ]:
import pandas as pd

# Adding condition labels
df_aerobic_plot = df_aerobic.copy()
df_aerobic_plot["Condition"] = "Aerobic"
df_aerobic_plot["Reaction"] = df_aerobic_plot.index

df_anaerobic_plot = df_anaerobic.copy()
df_anaerobic_plot["Condition"] = "Anaerobic"
df_anaerobic_plot["Reaction"] = df_anaerobic_plot.index

# Combining into one dataframe
df_plot = pd.concat([df_aerobic_plot, df_anaerobic_plot])


In [54]:
fig = px.bar(
    df_plot,
    x="Reaction",
    y="fluxes",
    color="Condition",
    barmode="group",
    title="Exchange fluxes under aerobic vs anaerobic conditions (Mouse-GEM)",
    template="none"
)

fig.update_layout(
    xaxis_tickangle=90,
    yaxis_title="Flux value"
)

fig.show()


In [25]:
#3.6 Analyse exchange fluxes (uptake and secretion)
#Even if growth is unchanged, metabolic inputs and outputs may differ

#Anaerobic exchange fluxes
df_aerobic = sol_aerobic.to_frame()
df_aerobic = df_aerobic.loc[[rxn.id for rxn in aerobic_model.exchanges]]
df_aerobic = df_aerobic[df_aerobic["fluxes"] != 0]

df_aerobic


,fluxes,reduced_costs
MAR09813,6.188417,0.0
MAR09816,0.417399,0.0
MAR09034,217.197646,0.0
MAR09035,-157.300000,0.0
MAR09036,-7.300000,0.0
...,...,...
MAR13063,-0.100000,0.0
MAR13067,-247.800000,0.0
MAR10024,4.262828,0.0
MAR10026,-160.738220,0.0


Analysis of exchange fluxes under aerobic conditions reveals that, despite unchanged growth rate, the model actively exchanges a large number of metabolites with the environment. Both uptake (negative fluxes) and secretion (positive fluxes) are observed, indicating extensive metabolic interaction with the medium. The presence of many non-zero exchange fluxes reflects the highly permissive default medium of Mouse-GEM. This highlights that growth alone does not capture the full metabolic response of the system to environmental conditions.

In [57]:
#Aneorobic exchange fluxes(O₂ uptake + CO₂ secretion blocked)

anaerobic_model = model.copy()

# Block CO2 secretion
anaerobic_model.reactions.get_by_id("MAR09058").upper_bound = 0

# Block O2 uptake
anaerobic_model.reactions.get_by_id("MAR09048").lower_bound = 0

solution_ana = anaerobic_model.optimize()

df_ana = solution_ana.to_frame()
df_ana = df_ana.loc[[rxn.id for rxn in anaerobic_model.exchanges]]
df_ana = df_ana[df_ana["fluxes"] != 0]

df_ana


,fluxes,reduced_costs
MAR09813,9.252283,0.0
MAR09034,239.473878,0.0
MAR09035,-36.319863,0.0
MAR09039,-18.146359,0.0
MAR09044,31.040981,0.0
...,...,...
MAR13059,-0.075864,0.0
MAR13063,-0.075864,0.0
MAR13067,-42.104489,0.0
MAR10024,4.262828,0.0


In [58]:
#3.7 Add readable reaction equations (for interpretation)
reaction_ids = df_ana.index

reaction_equations = {
    rid: anaerobic_model.reactions
        .get_by_id(rid)
        .build_reaction_string(use_metabolite_names=True)
    for rid in reaction_ids
}

df_ana["equations"] = reaction_equations
df_ana


,fluxes,reduced_costs,equations
MAR09813,9.252283,0.0,octanoic acid <=>
MAR09034,239.473878,0.0,glucose <=>
MAR09035,-36.319863,0.0,linoleate <=>
MAR09039,-18.146359,0.0,isoleucine <=>
MAR09044,31.040981,0.0,threonine <=>
...,...,...,...
MAR13059,-0.075864,0.0,nonadecylic acid <=>
MAR13063,-0.075864,0.0,ximenic acid <=>
MAR13067,-42.104489,0.0,retinyl palmitate <=>
MAR10024,4.262828,0.0,biomass <=>


Under anaerobic conditions with both oxygen uptake and carbon dioxide secretion blocked, the model exhibits a reduced set of active exchange reactions compared to the aerobic case. Although biomass production remains unchanged, the pattern and magnitude of uptake and secretion fluxes differ, indicating metabolic reorganisation. The decrease in the number of active exchanges reflects a more constrained metabolic state imposed by the environmental limitations. This demonstrates that environmental inputs can significantly affect metabolic flux distributions even when growth is unaffected.

In [59]:
#3.8 Visualise anaerobic exchange fluxes
import plotly.express as px

fig = px.bar(
    df_ana,
    x="equations",
    y="fluxes",
    template="none",
    title="Exchange fluxes under anaerobic conditions (Mouse-GEM)"
)

fig.show()


Figure X shows the exchange flux distribution under anaerobic conditions in the Mouse-GEM model. Positive fluxes correspond to metabolite secretion, while negative fluxes represent uptake. Blocking oxygen uptake altered the exchange profile, with multiple metabolites being secreted or consumed at varying rates. The presence of several exchange reactions operating at their bounds reflects the unconstrained nature of the generic model. Overall, this analysis demonstrates that environmental constraints influence predicted metabolic inputs and outputs, even when biomass production remains unchanged.

In [29]:
#3.9 Checking minimal medium with glucose + oxygen
#whether glucose alone (plus oxygen) can support biomass production

#Setting up helper functions
def set_medium(model, uptake_dict):
    # Close all uptake reactions
    for rxn in model.exchanges:
        rxn.lower_bound = 0

    # Open selected uptake reactions
    for rxn_id, lb in uptake_dict.items():
        model.reactions.get_by_id(rxn_id).lower_bound = lb


In [32]:
#Minimal medium with only glucose and oxygen
minimal_model = model.copy()

minimal_medium = {
    "MAR01947": -10,   # Alpha-D-glucose-1-phosphate
    "MAR09048": -20    # Oxygen
}

set_medium(minimal_model, minimal_medium)

sol_min = minimal_model.optimize()
print(f"Minimal medium growth rate: {sol_min.objective_value:.4f}")


Minimal medium growth rate: 0.0000


In [35]:
#Rich medium
rich_model = model.copy()

# Only limit glucose uptake, keep other exchanges open
rich_model.reactions.get_by_id("MAR01947").lower_bound = 0.2

sol_rich = rich_model.optimize()
print(f"Rich medium growth rate: {sol_rich.objective_value:.4f}")


Rich medium growth rate: 4.2628


In [36]:
sol_rich.fluxes["MAR01947"]


np.float64(0.2)

Under minimal medium conditions with glucose as the sole permitted carbon source, no biomass production was predicted, indicating that glucose alone is insufficient to sustain growth in the Mouse-GEM model. In contrast, when additional nutrient uptake was permitted in a rich medium, biomass production was restored to the baseline growth rate. This highlights the complex nutrient requirements of mammalian metabolism and demonstrates that biomass production in Mouse-GEM depends on the availability of multiple substrates rather than on glucose alone

In [61]:
#3.8 identify uptake reactions in the model
# Make sure objective is biomass
model.objective = "MAR00021"

# Baseline solution
baseline_solution = model.optimize()
baseline_growth = baseline_solution.objective_value

print(f"Baseline biomass growth: {baseline_growth:.4f}")

# Identify uptake reactions (negative lower bound)
uptake_reactions = [rxn for rxn in model.exchanges if rxn.lower_bound < 0]

print(f"Number of uptake reactions: {len(uptake_reactions)}")

# Inspect a few
uptake_reactions[:5]


Baseline biomass growth: 4.2628
Number of uptake reactions: 1658


[<Reaction MAR07108 at 0x1d889d02e10>,
 <Reaction MAR07110 at 0x1d889d02ed0>,
 <Reaction MAR07112 at 0x1d889d02f90>,
 <Reaction MAR07114 at 0x1d889d03050>,
 <Reaction MAR07116 at 0x1d889d03110>]

In [62]:
#3.9 Runnig a loop blocking each uptake reaction one at a time
import pandas as pd

uptake_effects = []

for rxn in uptake_reactions:
    
    # Save original bounds
    original_bounds = (rxn.lower_bound, rxn.upper_bound)
    
    # Block uptake
    rxn.lower_bound = 0
    
    # Run FBA
    solution = model.optimize()
    
    if solution.status == "optimal":
        biomass = solution.objective_value
    else:
        biomass = 0
    
    # Storing results
    uptake_effects.append({
        "rxn": rxn.id,
        "biomass_flux": biomass
    })
    
    # Restore bounds (CRITICAL)
    rxn.lower_bound, rxn.upper_bound = original_bounds

data = pd.DataFrame(uptake_effects)
data.head()


,rxn,biomass_flux
0,MAR07108,4.262828
1,MAR07110,4.262828
2,MAR07112,4.262828
3,MAR07114,4.262828
4,MAR07116,4.262828


In [63]:
#plotting the uptake and secretion fluxes
import plotly.express as px

default_biomass = model.optimize().objective_value

fig = px.bar(
    data,
    x="rxn",
    y="biomass_flux",
    template="none",
    title="Effect of blocking individual uptake reactions on biomass production"
)

fig.add_shape(
    type="line",
    x0=-0.5,
    x1=len(data.index) - 0.5,
    y0=default_biomass,
    y1=default_biomass,
    line=dict(color="red", dash="dash")
)

fig.show()


Blocking individual uptake reactions one at a time had no observable effect on biomass production, as the predicted growth rate remained identical to the baseline across all perturbations. This indicates that no single nutrient uptake reaction is essential for biomass formation under the default medium conditions. The result highlights the high metabolic redundancy and flexibility of the Mouse-GEM model. Biomass production is therefore supported by multiple alternative nutrient sources rather than relying on any single uptake reaction